<a href="https://colab.research.google.com/github/semreyalcin/CNG562-HW2/blob/master/YALCIN_malaria_cell_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q kaggle
!pip install -q kaggle-cli
!mkdir .kaggle
import json
token = {"username":"semizotu","key":"6671a08a833b87d14ce28d067f71f93a"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}    
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘.kaggle’: File exists
- path is now set to: {/content}


In [0]:
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria
!mv {/content}/datasets/iarunava/cell-images-for-detecting-malaria/cell-images-for-detecting-malaria.zip /content
!rm -r {/

 95% 319M/337M [00:02<00:00, 164MB/s]
100% 337M/337M [00:02<00:00, 131MB/s]


In [0]:
!unzip cell-images-for-detecting-malaria.zip

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


train = datagen.flow_from_directory("cell_images",
                                          target_size=(64, 64),
                                          batch_size=32,
                                          class_mode='binary',
                                          subset='training')

test = datagen.flow_from_directory( "cell_images",
                                         target_size=(64,64),
                                         batch_size=32,
                                         class_mode='binary',
                                         subset='validation')

Found 22048 images belonging to 2 classes.
Found 5510 images belonging to 2 classes.


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D

CNN = Sequential()

CNN.add(Conv2D(64, (3,3), activation = 'relu', input_shape = (64,64,3)))
CNN.add(MaxPool2D(pool_size=(2,2)))
CNN.add(Conv2D(32, (3,3), activation = 'relu'))
CNN.add(MaxPool2D(pool_size=(2,2)))
CNN.add(Flatten())
CNN.add(Dense(128, activation='relu'))
CNN.add(Dense(1, activation='softmax'))

In [0]:
CNN.compile(optimizer = 'adam', loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
CNN.fit_generator(train, steps_per_epoch = 8, epochs = 10, validation_data = test, validation_steps = 50)

Epoch 1/10
8/8 [==============================] - 4s 443ms/step - loss: 0.6990 - acc: 0.5117 - val_loss: 0.6920 - val_acc: 0.5032
Epoch 2/10
8/8 [==============================] - 2s 275ms/step - loss: 0.6918 - acc: 0.5391 - val_loss: 0.6926 - val_acc: 0.5044
Epoch 3/10
8/8 [==============================] - 2s 274ms/step - loss: 0.6946 - acc: 0.4961 - val_loss: 0.6963 - val_acc: 0.5038
Epoch 4/10
8/8 [==============================] - 2s 273ms/step - loss: 0.6964 - acc: 0.4648 - val_loss: 0.6928 - val_acc: 0.4994
Epoch 5/10
8/8 [==============================] - 2s 272ms/step - loss: 0.6913 - acc: 0.5859 - val_loss: 0.6913 - val_acc: 0.5273
Epoch 6/10
8/8 [==============================] - 2s 277ms/step - loss: 0.6913 - acc: 0.5156 - val_loss: 0.6949 - val_acc: 0.4963
Epoch 7/10
8/8 [==============================] - 2s 274ms/step - loss: 0.6805 - acc: 0.5352 - val_loss: 0.6852 - val_acc: 0.5456
Epoch 8/10
8/8 [==============================] - 2s 269ms/step - loss: 0.6688 - acc: 0.62

#Transfer Learning

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

tfl = Sequential()

tfl.add(ResNet50(weights='imagenet'))
tfl.add(Dense(1, activation = 'softmax'))

tfl.compile(optimizer = 'adam', loss='binary_crossentropy',metrics=['accuracy'])


train = datagen.flow_from_directory("cell_images",
                                          target_size=(224, 224),
                                          batch_size=32,
                                          class_mode='binary',
                                          subset='training')

test = datagen.flow_from_directory( "cell_images",
                                         target_size=(224, 224),
                                         batch_size=32,
                                         class_mode='binary',
                                         subset='validation')


tfl.fit_generator(train, steps_per_epoch = 8, epochs = 10, validation_data = test, validation_steps = 50)

Found 22048 images belonging to 2 classes.
Found 5510 images belonging to 2 classes.
Epoch 1/10
8/8 [==============================] - 31s 4s/step - loss: 8.4071 - acc: 0.4727 - val_loss: 8.1605 - val_acc: 0.4881
Epoch 2/10
8/8 [==============================] - 10s 1s/step - loss: 8.6562 - acc: 0.4570 - val_loss: 7.5826 - val_acc: 0.5244
Epoch 3/10
8/8 [==============================] - 10s 1s/step - loss: 7.2862 - acc: 0.5430 - val_loss: 8.2004 - val_acc: 0.4856
Epoch 4/10
8/8 [==============================] - 10s 1s/step - loss: 7.7844 - acc: 0.5117 - val_loss: 7.5863 - val_acc: 0.5241
Epoch 5/10
8/8 [==============================] - 10s 1s/step - loss: 7.7844 - acc: 0.5117 - val_loss: 8.0708 - val_acc: 0.4938
Epoch 6/10
8/8 [==============================] - 10s 1s/step - loss: 8.2203 - acc: 0.4844 - val_loss: 8.0908 - val_acc: 0.4925
Epoch 7/10
8/8 [==============================] - 10s 1s/step - loss: 8.4071 - acc: 0.4727 - val_loss: 8.1940 - val_acc: 0.4860
Epoch 8/10
8/8 [===

For some reason I couldn't train the models well. They did not improve their results in each epoch.